In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

import cv2
import random
import tables
import openslide
from misc.utils import extract_patch, draw_circle,is_nucleus

import torch
import logging
import os
import copy

import torch
import logging
import os
import copy
from misc.utils import log_info


In [2]:

#Logging
logging.basicConfig(
    level=logging.INFO,
    format='|%(asctime)s.%(msecs)03d| [%(levelname)s] %(message)s',datefmt='%Y-%m-%d|%H:%M:%S',
    handlers=[
        logging.FileHandler("debug.log"),
        logging.StreamHandler()
    ]
)

In [3]:
HE_path = 'wsi/MELANOMA_AA3_40X_Scan1.qptiff'
HE_open = openslide.OpenSlide(HE_path)
HE_open.properties['tiff.XResolution']

'40011.246972350571'

In [4]:
# Checking for GPU
gpu_list = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_list
nr_gpus = torch.cuda.device_count()
log_info('Detect #GPUS: %d' % nr_gpus)

|2021-09-14|16:59:14.303| [INFO]  Detect #GPUS: 1


In [5]:
method_args = {
    'method' : {
        'model_args' : {
            'nr_types'   : None,
            'mode'       : 'original',
        },
        'model_path' : 'pretrained/hovernet_original_consep_notype_tf2pytorch.tar',
    },
    'type_info_path'  : None,
}

In [6]:
run_args = {
    'batch_size' : 8 * nr_gpus,
    'nr_inference_workers' : 8,
    'nr_post_proc_workers' : 16,
    'patch_input_shape': 270,
    'patch_output_shape' : 80,
    'input_dir'      : './wsi/',
    'output_dir'     : './hovernet_output/',
    'cache_path'     : 'cache',
    'input_mask_dir' : '',
    'proc_mag'       : 40,
    'ambiguous_size' : 128,
    'chunk_shape'    : 10000,
    'tile_shape'     : 2048,
    'save_thumb'     : False,
    'save_mask'      : True,
}

In [7]:
from infer.wsi import InferManager
infer = InferManager(**method_args)
infer.process_wsi_list(run_args)


|2021-09-14|16:59:17.871| [INFO] ................ Process: MELANOMA_AA3_40X_Scan1
|2021-09-14|16:59:17.874| [ERROR] Crash
Traceback (most recent call last):
  File "/data/home/jmapanao/mIF2HE/infer/wsi.py", line 739, in process_wsi_list
    self.process_single_file(wsi_path, msk_path, self.output_dir)
  File "/data/home/jmapanao/mIF2HE/infer/wsi.py", line 466, in process_single_file
    read_mag=self.proc_mag, cache_path="%s/src_wsi.npy" % self.cache_path
  File "/data/home/jmapanao/mIF2HE/misc/wsi_handler.py", line 72, in prepare_reading
    np.save(cache_path, self.get_full_img(read_mag=read_mag))
  File "/data/home/jmapanao/mIF2HE/misc/wsi_handler.py", line 182, in get_full_img
    wsi_img = self.file_ptr.read_region((0, 0), read_lv, read_size)
  File "/data/home/jmapanao/mIF2HE/mIF2HE_env/lib/python3.6/site-packages/openslide/__init__.py", line 229, in read_region
    level, size[0], size[1])
  File "/data/home/jmapanao/mIF2HE/mIF2HE_env/lib/python3.6/site-packages/openslide/lowlev